# Test the performance of the GPU/CPU:

## Torch

In [1]:
import torch


def create_torch_tensors(device):
    x = torch.rand((10000, 10000), dtype=torch.float32)
    y = torch.rand((10000, 10000), dtype=torch.float32)
    x = x.to(device)
    y = y.to(device)

    return x, y


In [2]:
device = torch.device("cpu")
x, y = create_torch_tensors(device)


In [3]:
%%timeit
x * y

19.8 ms ± 520 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
device = torch.device("mps")
x, y = create_torch_tensors(device)


In [5]:
%%timeit
x * y


9.49 ms ± 51.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Tensorflow

In [2]:
import tensorflow as tf


def create_tf_tensors():
    x = tf.random.uniform((10000, 10000), dtype=tf.float32)
    y = tf.random.uniform((10000, 10000), dtype=tf.float32)

    return x, y


x, y = create_tf_tensors()

In [3]:
%%timeit

with tf.device("/CPU:0"):
    x * y

21.2 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit

with tf.device("/GPU:0"):
    x * y

9.68 ms ± 65.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## JAX

In [5]:
import jax
import jax.numpy as jnp


def create_jax_tensors():
    x = jax.random.uniform(jax.random.PRNGKey(0), (10000, 10000), dtype=jnp.float32)
    y = jax.random.uniform(jax.random.PRNGKey(1), (10000, 10000), dtype=jnp.float32)

    return x, y


x, y = create_jax_tensors()

objc[40024]: Class MetalStreamWrapper is implemented in both /Users/yalhari/apple_silicon/.venv/lib/python3.11/site-packages/tensorflow-plugins/libmetal_plugin.dylib (0x375a49638) and /Users/yalhari/apple_silicon/.venv/lib/python3.11/site-packages/jax_plugins/metal_plugin/pjrt_plugin_metal_14.dylib (0x4072991f8). One of the two will be used. Which one is undefined.
Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!
W0000 00:00:1716816308.432635  352963 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1716816308.433327  352963 service.cc:145] XLA service 0x105fdbee0 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716816308.433344  352963 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1716816308.434240  352963 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1716816308.434245  35

Metal device set to: Apple M3 Pro

systemMemory: 18.00 GB
maxCacheSize: 6.00 GB



In [6]:
%%timeit
x * y

9.56 ms ± 31.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now, you need to restart the kernel, so the following lines executed in the cpu

In [4]:
import os

os.environ["JAX_PLATFORMS"] = "cpu"

In [5]:
import jax
import jax.numpy as jnp


def create_jax_tensors():
    x = jax.random.uniform(jax.random.PRNGKey(0), (10000, 10000), dtype=jnp.float32)
    y = jax.random.uniform(jax.random.PRNGKey(1), (10000, 10000), dtype=jnp.float32)

    return x, y


x, y = create_jax_tensors()

In [6]:
%%timeit
x * y

16.8 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Source:
https://github.com/svpino/apple-silicon/blob/main/test.ipynb